In [ ]:
#hide
#default_exp config.manager_factory
from nbdev.showdoc import show_doc
import warnings
warnings.filterwarnings('ignore')

# Manager factory

> Register a subclassed ExperimentManager to be used by other modules. 

In [ ]:
#export
import inspect
import shutil
import hpsearch
import os
import logging

experiment_manager = None

class ManagerFactory (object):
    def __init__ (self, allow_base_class=True):
        self.allow_base_class = allow_base_class
        self.logger = logging.getLogger("experiment_manager")
        
    def register_manager (self, experiment_manager_to_register):
        global experiment_manager
        experiment_manager = experiment_manager_to_register
        
    def get_destination_path (self):
        destination_path_folder = os.path.dirname (hpsearch.__file__)
        destination_path_folder = f'{destination_path_folder}/app_config'
        destination_path_module = f'{destination_path_folder}/subclassed_manager.py'
        return destination_path_folder, destination_path_module
        
    def write_manager (self, experiment_manager):
        name_subclass = experiment_manager.__class__.__name__
        try:
            source_path = inspect.getfile(experiment_manager.__class__)

            destination_path_folder, destination_path_module = self.get_destination_path()
            os.makedirs(destination_path_folder, exist_ok=True)
            shutil.copy (source_path, destination_path_module)

            f = open (f'{destination_path_folder}/subclassed_manager_import.py', 'wt')
            f.write (f'from hpsearch.app_config.subclassed_manager import {name_subclass} as Manager')
            f.close()
        except Exception as e:
            print (f'write_manager failed with exception {e}')
        
    def get_experiment_manager (self):
        if experiment_manager is not None:
            em = experiment_manager
            self.logger.debug ('returning registered experiment manager')
        else:
            self.logger.debug ('experiment manager not registered yet, importing experiment manager')
            try: 
                from hpsearch.app_config.subclassed_manager_import import Manager
                em = Manager()
                self.logger.debug ('returning subclassed manager')
            except ImportError:
                if not self.allow_base_class:
                    raise ImportError (f'it was not possible to import subclassed manager, and allow_base_class=False')
                self.logger.debug ('importing base class ExperimentManager')
                from hpsearch.experiment_manager import ExperimentManager
                em = ExperimentManager()
        
        self.logger.debug (f'returning experiment manager {em}')
        return em
    
    def delete_module (self):
        destination_path_folder, destination_path_module = self.get_destination_path()
        self.logger.debug (destination_path_module)
        if os.path.exists(destination_path_module):
            self.logger.debug ('deleting')
            os.remove(destination_path_module)

The method `get_experiment_manager` returns the base class ExperimentManager initially, unless a subclass definition has been stored by calling the `write_manager` method

In [ ]:
factory = ManagerFactory ()
factory.delete_module()
em = factory.get_experiment_manager()
assert em.__class__.__name__ == 'ExperimentManager'

We can write the definition of a desired subclass of ExperimentManager with the method `write_manager`. After writing this definition, the object manager_factory will return an instance of the same subclass for any module that imports ManagerFactory. Let us check this with one example. We will use the subclass `ExampleExperimentManager` as an example, and check that the module file exists calling the method `write_manager`.

In [ ]:
from hpsearch.examples.example_experiment_manager import ExampleExperimentManager

# we check that the file does not exist before writting it
destination_path_folder, destination_path_module = factory.get_destination_path()
assert not os.path.exists(destination_path_module)

# we write new module and check that the file exists afterwards
em = ExampleExperimentManager()
factory.write_manager (em)
assert os.path.exists(destination_path_module)

# we check that the written module corresponds to the class ExampleExperimentManager
from hpsearch.app_config.subclassed_manager_import import Manager
em2 = Manager()
assert Manager.__name__ == 'ExampleExperimentManager'
assert em2.__class__.__name__ == em.__class__.__name__
assert em.get_default_parameters({}) == em2.get_default_parameters({})

After writting the subclass, the method `get_experiment_manager` returns an instance of this subclass. This allows command-line scripts to use functions of the subclass indicated by other modules. 

In [ ]:
em2 = factory.get_experiment_manager()
assert em2.__class__.__name__ == em.__class__.__name__

The method `get_experiment_manager` returns a registered experiment_manager

In [ ]:
from hpsearch.experiment_manager import ExperimentManager
em = ExperimentManager()
factory.register_manager(em)
em2 = factory.get_experiment_manager()
assert em2.__class__.__name__ == 'ExperimentManager'

em = ExampleExperimentManager()
factory.register_manager(em)
em2 = factory.get_experiment_manager()
assert em2.__class__.__name__ == 'ExampleExperimentManager'

factory.register_manager(None)
em2 = factory.get_experiment_manager()
assert em2.__class__.__name__ == 'ExampleExperimentManager'

In [ ]:
#hide
from nbdev.export import *
notebook2script(recursive=True)

Converted hpconfig.ipynb.
Converted manager_factory.ipynb.
Converted example_experiment.ipynb.
Converted example_experiment_manager.ipynb.
Converted experiment_manager.ipynb.
Converted index.ipynb.
Converted experiment_utils.ipynb.
Converted organize_experiments.ipynb.


We check that a module registered in one module is used in another one. 

In [ ]:
del ManagerFactory
del experiment_manager
from hpsearch.config.manager_factory import ManagerFactory

factory = ManagerFactory()
em = factory.get_experiment_manager()
assert em.__class__.__name__ == 'ExampleExperimentManager'

defaults=em.get_default_parameters({})

We check that the defaults are removed properly by using remove_defaults

In [ ]:
from hpsearch.utils.experiment_utils import remove_defaults

removed=remove_defaults(defaults)
assert removed=={}

removed=remove_defaults({'my_first':50, 'my_second':10, 'my_third':30})
assert removed=={'my_first': 50, 'my_third': 30}
removed

{'my_first': 50, 'my_third': 30}

We check that no defaults are removed if the base class ExperimentManager is registered

In [ ]:
em3 = ExperimentManager(allow_base_class=True)
factory.register_manager(em3)
parameters = {'my_first':50, 'my_second':10, 'my_third':30}
parameters_copy = parameters.copy()
removed=remove_defaults(parameters_copy)
assert removed==parameters
removed

{'my_first': 50, 'my_second': 10, 'my_third': 30}